## Quantifying Linguistic Degeneration

Once I have the data, I must think about how to quantify linguistic degeneration. For this, I will be using the following metrics:
<br>

***Lexical Quality***

1. Type-token ratio (TTR): measures vocabulary diversity
2. Zipf curve tail heaviness: measures use of rare words
3. Average word length: measures use of larger words
4. Age of acquisition: measures word difficulty

***Syntactic Quality***

5. Parse tree depth: measures sentence complexity
6. Fragment ratio: measure of informality
7. Index of Syntactic Complexity: measures overall syntactic quality

***Orthographical Quality***

8. Levenshtein distance to dictionary: spelling errors
9. Punctuation frequency: attention to grammar

***Substantive Quality***

10. Abstract concepts
11. Figurative language

Note that I do not use other traditional measures of lexical complexity, such as ambiguity, vague quantifier frequency, orthographic neighborhood size, terminology inconsistency. Word ambiguity almost exclusively measures the complexity of the text from the perspective of the reader. As the purpose of this analysis is to look at language degeneration over time as a result of modern phenomena, I am principally interested in the intellect required to produce the complexity, meaning I am looking at true substantive complexity as opposed to inteprative complexity. Similarly, vague quantifier frequency, orthographic neighborhood size, and terminology inconsistency also measure complexity of interpratation, hence they are excluded. It is also worth noting that I categorize abstract concepts and figurative language under substantive quality as opposed to lexical complexity, as they involve more intimately the meaning of the word.

Likewise, I exclude certain measures of syntactic complexity, such as sentence length, information overload, passive voice, and negation. In the case of sentence length, it can be a sign of high (i.e., complexity) or low (i.e., run-on sentences) quality writing, which is why I have excluded it. Additionally, sentence complexity can be better measured via ICS. Information overload, passive voice, and negation are excluded for the same interpratability versus quality distinction in the above paragraph.

## Computing Lexical Quality Metrics With Python

In [66]:
import re
import numpy as np
import pandas as pd
from wordfreq import zipf_frequency

In [67]:
# sample post from the Cornell subreddit for testing tokenizers
cornell_example = "Based on 2017 numbers:    \n\n3375 entering freshman, 56.6% yield, meaning 5962 were accepted out of 47039. That gives a 12.7% acceptance rate.    \n\nIf Cornell were to enroll 900 more students, that'd be 225 additional students per year.  That works out to (3375+225)/0.566 = 6360 accepted students, giving a theoretical acceptance rate of 13.5% if Cornell had implemented this change in 2017.    \n\nKeep in mind that this is not an accurate projection for 2021 because we get ~2000 more applicants each year, so acceptance rates will actually continue to fall.   \n\nRegardless, a 0.8% rise in acceptance doesn't seem too bad.  As long as the faculty can handle the moderate increase in class sizes and the quality of education stays the same, I don't see a reason to reject more people than we have to.    \n\nSource: http://irp.dpb.cornell.edu/tableau_visual/admissions"

In [93]:
def simple_tokenize(text):
    '''Helper function to tokenize a string of text, removing non-alphabetic characters'''
    
    text = text.lower()
    processed = re.sub(r'[^a-zA-Z\s]', '', text)
    tokens = processed.split()

    return tokens

In [94]:
example = "Nicholas $%#! @@is# $@! 123 gre4at.👍"
print(example)
print(simple_tokenize(example))
print('The result should only contain words\n')
print(simple_tokenize(cornell_example))

Nicholas $%#! @@is# $@! 123 gre4at.👍
['nicholas', 'is', 'great']
The result should only contain words

['based', 'on', 'numbers', 'entering', 'freshman', 'yield', 'meaning', 'were', 'accepted', 'out', 'of', 'that', 'gives', 'a', 'acceptance', 'rate', 'if', 'cornell', 'were', 'to', 'enroll', 'more', 'students', 'thatd', 'be', 'additional', 'students', 'per', 'year', 'that', 'works', 'out', 'to', 'accepted', 'students', 'giving', 'a', 'theoretical', 'acceptance', 'rate', 'of', 'if', 'cornell', 'had', 'implemented', 'this', 'change', 'in', 'keep', 'in', 'mind', 'that', 'this', 'is', 'not', 'an', 'accurate', 'projection', 'for', 'because', 'we', 'get', 'more', 'applicants', 'each', 'year', 'so', 'acceptance', 'rates', 'will', 'actually', 'continue', 'to', 'fall', 'regardless', 'a', 'rise', 'in', 'acceptance', 'doesnt', 'seem', 'too', 'bad', 'as', 'long', 'as', 'the', 'faculty', 'can', 'handle', 'the', 'moderate', 'increase', 'in', 'class', 'sizes', 'and', 'the', 'quality', 'of', 'education

Notice that the simple tokenizer cannot adequately handle links. See the last index: "httpirpdpbcornelledutableauvisualadmissions"

In [80]:
# set up nltk tokenizer
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize, TweetTokenizer
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/nickvick/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [70]:
print('Now try the advanced tokenizer')
another_example = "Nicholas is great. Hopefully this works. I now have hope. 👍."
print(another_example)
print(word_tokenize(another_example))

Now try the advanced tokenizer
Nicholas is great. Hopefully this works. I now have hope. 👍.
['Nicholas', 'is', 'great', '.', 'Hopefully', 'this', 'works', '.', 'I', 'now', 'have', 'hope', '.', '👍', '.']


In [95]:
def tokenize(text):
    '''Helper function to tokenize social media text. Note that the TweetTokenizer 
    preserves mentions, contractions'''
    tokenizer = TweetTokenizer()
    tokens = tokenizer.tokenize(text)

    return tokens

In [96]:
print("Let's try this on a line from the actual Cornell subreddit")
tokenize(cornell_example)

Let's try this on a line from the actual Cornell subreddit


['Based',
 'on',
 '2017',
 'numbers',
 ':',
 '3375',
 'entering',
 'freshman',
 ',',
 '56.6',
 '%',
 'yield',
 ',',
 'meaning',
 '5962',
 'were',
 'accepted',
 'out',
 'of',
 '47039',
 '.',
 'That',
 'gives',
 'a',
 '12.7',
 '%',
 'acceptance',
 'rate',
 '.',
 'If',
 'Cornell',
 'were',
 'to',
 'enroll',
 '900',
 'more',
 'students',
 ',',
 "that'd",
 'be',
 '225',
 'additional',
 'students',
 'per',
 'year',
 '.',
 'That',
 'works',
 'out',
 'to',
 '(',
 '3375',
 '+',
 '225',
 ')',
 '/',
 '0.566',
 '=',
 '6360',
 'accepted',
 'students',
 ',',
 'giving',
 'a',
 'theoretical',
 'acceptance',
 'rate',
 'of',
 '13.5',
 '%',
 'if',
 'Cornell',
 'had',
 'implemented',
 'this',
 'change',
 'in',
 '2017',
 '.',
 'Keep',
 'in',
 'mind',
 'that',
 'this',
 'is',
 'not',
 'an',
 'accurate',
 'projection',
 'for',
 '2021',
 'because',
 'we',
 'get',
 '~',
 '2000',
 'more',
 'applicants',
 'each',
 'year',
 ',',
 'so',
 'acceptance',
 'rates',
 'will',
 'actually',
 'continue',
 'to',
 'fall',
 '

In [97]:
def clean_lexical_tokens(tokens):
    '''Helper function to clean tokens by removing punctuation, numbers, and emojis
    for purely lexical analysis.'''

    cleaned = []

    for tok in tokens:
        # skip over punctuation
        if re.match(r'^\W+$', tok):
            continue
        # skip over emojis
        # if tok.encode()
        # only keep alphabetic tokens
        if tok.isalpha():
            cleaned.append(tok.lower())

    return cleaned

In [98]:
clean_lexical_tokens(tokenize(cornell_example))

['based',
 'on',
 'numbers',
 'entering',
 'freshman',
 'yield',
 'meaning',
 'were',
 'accepted',
 'out',
 'of',
 'that',
 'gives',
 'a',
 'acceptance',
 'rate',
 'if',
 'cornell',
 'were',
 'to',
 'enroll',
 'more',
 'students',
 'be',
 'additional',
 'students',
 'per',
 'year',
 'that',
 'works',
 'out',
 'to',
 'accepted',
 'students',
 'giving',
 'a',
 'theoretical',
 'acceptance',
 'rate',
 'of',
 'if',
 'cornell',
 'had',
 'implemented',
 'this',
 'change',
 'in',
 'keep',
 'in',
 'mind',
 'that',
 'this',
 'is',
 'not',
 'an',
 'accurate',
 'projection',
 'for',
 'because',
 'we',
 'get',
 'more',
 'applicants',
 'each',
 'year',
 'so',
 'acceptance',
 'rates',
 'will',
 'actually',
 'continue',
 'to',
 'fall',
 'regardless',
 'a',
 'rise',
 'in',
 'acceptance',
 'seem',
 'too',
 'bad',
 'as',
 'long',
 'as',
 'the',
 'faculty',
 'can',
 'handle',
 'the',
 'moderate',
 'increase',
 'in',
 'class',
 'sizes',
 'and',
 'the',
 'quality',
 'of',
 'education',
 'stays',
 'the',
 's

In [99]:
def ttr(text):
    '''Function that returns the type-token ratio'''

    tokens = tokenize(text)
    tokens = clean_lexical_tokens(tokens)

    # error handling for when there are no tokens
    if len(tokens) == 0:
        return 0.0
    
    # recall that TTR is number of unique words / number of words
    num_types = len(set(tokens))
    num_tokens = len(tokens)
    ttr = num_types / num_tokens

    return ttr

In [100]:
example2 = "Nicholas is Nicholas"
print(ttr(example2))
print('The result should be 0.66')

0.6666666666666666
The result should be 0.66


In [101]:
def avg_word_length(text):
    '''Function that determines the average word length of a given text'''
    
    tokens = tokenize(text)
    tokens = clean_lexical_tokens(tokens)

    # error handling for when there are no tokens
    if len(tokens) == 0:
        return 0.0
    
    average_length = np.mean([len(word) for word in tokens])

    return average_length

In [102]:
example6 = "\nNicholas is great 👍 &*&()"
print(example6)
print(tokenize(example6))
print(clean_lexical_tokens(tokenize(example6)))

print(avg_word_length(example6))
print('The result should be 5')


Nicholas is great 👍 &*&()
['Nicholas', 'is', 'great', '👍', '&', '*', '&', '(', ')']
['nicholas', 'is', 'great']
5.0
The result should be 5


In [103]:
# build aoa_dict: word -> average age of acquisition
aoa_df = pd.read_csv("Data/KupermanAoAData.csv")
aoa_dict = dict(zip(aoa_df["word"], aoa_df["rating_mean"]))

def aoa_score(text, aoa_dict):
    '''Returns the average age of acquisition score for a given text'''
    
    tokens = tokenize(text)
    tokens = clean_lexical_tokens(tokens)
    aoa_values = [aoa_dict[word] for word in tokens if word in aoa_dict]

    # if there are no words, return a default value
    if len(aoa_values) == 0:
        return np.nan
    
    aoa_score = np.mean(aoa_values)

    return aoa_score

In [91]:
example3 = "because I am cool"
print(aoa_score(example3, aoa_dict))
print('This result should be lower than:')
example4 = "sophisticated technical jargon"
print(aoa_score(example4, aoa_dict))

4.51
This result should be lower than:
11.163333333333334


In [88]:
def zipf_score(text):
    '''Returns a frequency score (higher -> more frequent) based on the Zipf scale'''
    
    tokens = tokenize(text)
    tokens = clean_lexical_tokens(tokens)
    
    zipf_values = [zipf_frequency(word, 'en') for word in tokens]

     # if there are no words, return a default value
    if len(zipf_values) == 0:
        return np.nan

    zipf_score = np.mean(zipf_values)

    return zipf_score

In [89]:
print(zipf_score(example3))
print('This result should be higher than:')
print(zipf_score(example4))
print()

example5 = "word"
print(zipf_score(example5))
print('This result should be 5.26')

6.012500000000001
This result should be higher than:
4.013333333333334

5.26
This result should be 5.26


## Computing Syntactic Quality Metrics With Python

Parse-tree depth

Fragment ratio

Index of Syntactic Complexity

Note that in order to run the following text block, the following must be run to install the relevant NLP model:
<br>
**python3 -m spacy download en_core_web_sm**

In [112]:
import spacy
nlp = spacy.load("en_core_web_sm") # pre-trained English model

from nltk import pos_tag
import language_tool_python

In [86]:
def split_sentences(text):
    '''Helper function to split a given post into separate sentences'''

    sentence_tokens = sent_tokenize(text)

    return sentence_tokens

In [92]:
print(cornell_example)
print('-----------------------')

split_result = split_sentences(cornell_example)
print(split_result)
print()
for sent in split_result:
    print(sent)

Based on 2017 numbers:    

3375 entering freshman, 56.6% yield, meaning 5962 were accepted out of 47039. That gives a 12.7% acceptance rate.    

If Cornell were to enroll 900 more students, that'd be 225 additional students per year.  That works out to (3375+225)/0.566 = 6360 accepted students, giving a theoretical acceptance rate of 13.5% if Cornell had implemented this change in 2017.    

Keep in mind that this is not an accurate projection for 2021 because we get ~2000 more applicants each year, so acceptance rates will actually continue to fall.   

Regardless, a 0.8% rise in acceptance doesn't seem too bad.  As long as the faculty can handle the moderate increase in class sizes and the quality of education stays the same, I don't see a reason to reject more people than we have to.    

Source: http://irp.dpb.cornell.edu/tableau_visual/admissions
-----------------------
['Based on 2017 numbers:    \n\n3375 entering freshman, 56.6% yield, meaning 5962 were accepted out of 47039.'

Note that the following must be run in order to use LanguageTool:
<br>

**brew install openjdk@17**

**sudo ln -sfn $(brew --prefix)/opt/openjdk@17/libexec/openjdk.jdk /Library/Java/JavaVirtualMachines/openjdk-17.jdk**

In [114]:
tool = language_tool_python.LanguageTool('en-US')

In [79]:
# define relevant sets of tags
finite_verb_tags = {"VB", "VBD", "VBN", "VBP", "VBZ"}
subject_tags = {"NN", "NNS", "NNP", "NNPS", "PRP"}

coordinating_comj = {}


def is_complete_sentence(sentence):
    '''Helper function to determine whether a sentence is complete. Recall that a complete sentence follows these rules:
    -contains at least one subject 
    -contains at least one finite verb
    -ends with appropriate punctuation (.?!) 
    -does not end with a conjunction
    -if it begins with a subordinator, has an independent clause after
    '''

    tokens = tokenize(sentence)
    tags = pos_tag(tokens)

    # first word should be capital

    # should encounter a noun first, then a finite verb


    # does not end with a conjunction
    if tokens[-1] 



for sent in doc.sents:
    if sent[0].is_uppercase and sent[-1].is_punct:
        has_noun = 2
        has_verb = 1
        for token in sent:
            if token.pos_ in ["NOUN", "PROPN", "PRON"]:
                has_noun -= 1
            elif token.pos_ == "VERB":
                has_verb -= 1
         if has_noun < 1 and has_verb < 1:
             print(sent.string.strip())

IndentationError: unindent does not match any outer indentation level (<string>, line 26)